In [1]:
# Librerías
import streamlit as st 
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import statsmodels.api as sm 
from statsmodels.formula.api import ols
from scipy import stats

In [ ]:
# Revisar DF
df = pd.read_csv('Berlin_86.csv')
berlin = df.drop(['Unnamed: 0','latitude', 'longitude'], axis=1)
berlin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14187 entries, 0 to 14186
Data columns (total 84 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   listing_url                                   14187 non-null  object 
 1   last_scraped                                  14187 non-null  object 
 2   source                                        14187 non-null  object 
 3   name                                          14187 non-null  object 
 4   description                                   14187 non-null  object 
 5   neighborhood_overview                         14187 non-null  object 
 6   picture_url                                   14187 non-null  object 
 7   host_id                                       14187 non-null  int64  
 8   host_url                                      14187 non-null  object 
 9   host_name                                     14187 non-null 

In [ ]:
# Chequeo de Null
berlin.isnull().sum()

listing_url                 0
last_scraped                0
source                      0
name                        0
description                 0
                           ..
host_response_rate_cat      0
review_scores_rating_cat    0
availability_365_cat        0
price_cat                   0
beds_cat                    0
Length: 84, dtype: int64

In [5]:
# Ajustar tipos de variables
berlin['host_id'] = berlin['host_id'].astype(str)

In [ ]:
%%writefile app.py
# Se crea el archivo de la APP en el interprete principal (Phyton)
##########
# Importar librerías
import streamlit as st 
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import statsmodels.api as sm 
from statsmodels.formula.api import ols
from scipy import stats

# Definir instancia
@st.cache_resource
#####

# Crear función de carga de datos
def load_data():
    df = pd.read_csv('Berlin_86.csv')
    # Relleno de nulos
    df = df.fillna(method= 'bfill')
    df = df.fillna(method= 'ffill')
    # Lista
    Lista = ['Survived', 'Pclass', 'Sex', 'Age']
    return df, Lista
####
# Carga de datos función 'load_data()'
df, Lista = load_data()

####
# Creación del dashboard
# Generar las páginas a utiizar en el diseño
####
# Generamos encabezados para barra lateral (sidebar)
st.sidebar.title('TITANIC')

# Widget 1: Selectbox
# Menú desplegable de opciones de las páginas seleccionadas
View = st.sidebar.selectbox(label= 'Tipo de análisis', options= ['Extracción de Características', 'Regresión Lineal', 'Regresión No Lineal', 'Regresión Logística'])

# CONTENIDO DE LA VISTA 1
if View == "Extracción de Características":
    #EXTRACCIÓN DE CARACTERÍSTICAS
    #Select box
    Variable_Cat= st.sidebar.selectbox(label= "Variables", options= Lista)
    #Obtenemos las frecuencias de las categorías de la variable seleccionada
    Tabla_frecuencias = df[Variable_Cat].value_counts().reset_index()
    #Ajustamos los nombre de las cabeceras de las columnas
    Tabla_frecuencias.columns = ['categorias', 'frecuencia']
    
    #Generamos los encabezados para el dashboard
    st.title("Extracción de Características")
    
    #Generamos el diseño del Layout deseado
    # Fila 1
    Contenedor_A, Contenedor_B = st.columns(2)
    with Contenedor_A: 
        st.write("Grafico de Barras")
        #GRAPH 1: BARPLOT
        #Despliegue de un bar plot, definiendo las variables "X categorias" y "Y numéricas" 
        figure1 = px.bar(data_frame=Tabla_frecuencias, x='categorias', 
                  y= 'frecuencia', title= str('Frecuencia por categoría'))
        figure1.update_xaxes(automargin=True)
        figure1.update_yaxes(automargin=True)
        figure1.update_layout(height=300)
        st.plotly_chart(figure1, use_container_width=True)

    with Contenedor_B:
        st.write("Grafico de Pastel")
        #GRAPH 2: PIEPLOT
        #Despliegue de un pie plot, definiendo las variables "X categorias" y "Y numéricas" 
        figure2 = px.pie(data_frame=Tabla_frecuencias, names='categorias', 
                  values= 'frecuencia', title= str('Frecuencia por categoría'))
        figure2.update_layout(height=300)
        st.plotly_chart(figure2, use_container_width=True)

    # Fila 2
    Contenedor_C, Contenedor_D = st.columns(2)
    with Contenedor_C:
        st.write("Grafico de anillo o dona")
        #GRAPH 3: DONUT PLOT
        #Despliegue de un line plot, definiendo las variables "X categorias" y "Y numéricas" 
        figure3 = px.pie(data_frame=Tabla_frecuencias, names='categorias', 
                  values= 'frecuencia', hole=0.4, title= str('Frecuencia por categoría'))
        figure3.update_layout(height=300)
        st.plotly_chart(figure3, use_container_width=True)

    with Contenedor_D:  
        st.write("Grafico de area")
        #GRAPH 4: AREA PLOT
        #Despliegue de un area plot, definiendo las variables "X categorias" y "Y numéricas" 
        figure4 = px.area(data_frame=Tabla_frecuencias, x='categorias', 
                  y= 'frecuencia', title= str('Frecuencia por categoría'))
        figure4.update_layout(height=300)
        st.plotly_chart(figure4, use_container_width=True)
 ############################################################################

  # CONTENIDO DE LA VISTA 2
if View == "Regresión Lineal":
    #REGRESIÓN LINEAL SIMPLE 
    #Generamos la lista de variables numéricas
    numeric_df = df.select_dtypes(['float','int'])  #Devuelve Columnas
    Lista_num= numeric_df.columns                   #Devuelve lista de Columnas numéricas
    #Select box
    Variable_y= st.sidebar.selectbox(label= "Variable objetivo (Y)", options= Lista_num)
    Variable_x= st.sidebar.selectbox(label= "Variable independiente del modelo simple (X)", options= Lista_num)    
    
    
    #Generamos los encabezados para el dashboard
    st.title("Regresión Lineal")  
    
    #Generamos el diseño del Layout deseado
    # Fila 1
    Contenedor_A, Contenedor_B = st.columns(2)
    with Contenedor_A: 
        st.write("Correlación Lineal Simple")
      
        #Se define model como la función de regresión lineal
        from sklearn.linear_model import LinearRegression
        model= LinearRegression()
        #Ajustamos el modelo con las variables antes declaradas
        model.fit(X=df[[Variable_x]], y=df[Variable_y])
        #Predecimos los valores de la variable objetivo
        y_pred= model.predict(X=df[[Variable_x]])
        #Corroboramos cual es el coeficiente de Determinación de nuestro modelo
        coef_Deter_simple=model.score(X=df[[Variable_x]], y=df[Variable_y])
        #Corroboramos cual es el coeficiente de Correlación de nuestro modelo
        coef_Correl_simple=np.sqrt(coef_Deter_simple)
        #Mostramos el dataset
        st.write(coef_Correl_simple)
        
        #GRAPH 5: SCATTERPLOT
        figure5 = px.scatter(data_frame=numeric_df, x=Variable_x, y=Variable_y, 
                     title= 'Modelo Lineal Simple')
        st.plotly_chart(figure5)
    

    with Contenedor_B:
        st.write("Correlación Lineal Múltiple")
        #Widget 3: Multiselect box
        #Generamos un cuadro de multi-selección (X) para seleccionar variables independientes
        Variables_x= st.sidebar.multiselect(label="Variables independientes del modelo múltiple (X)", options= Lista_num)
        #Se define model como la función de regresión lineal
        from sklearn.linear_model import LinearRegression
        model_M= LinearRegression()
        #Ajustamos el modelo con las variables antes declaradas
        model_M.fit(X=df[Variables_x], y=df[Variable_y])
        #Predecimos los valores de la variable objetivo
        y_pred_M= model_M.predict(X=df[Variables_x])
        #Corroboramos cual es el coeficiente de Determinación de nuestro modelo
        coef_Deter_multiple=model_M.score(X=df[Variables_x], y=df[Variable_y])
        #Corroboramos cual es el coeficiente de Correlación de nuestro modelo
        coef_Correl_multiple=np.sqrt(coef_Deter_multiple)
        #Mostramos el coeficiente de correlación múltiple
        st.write(coef_Correl_multiple)
        
        #GRAPH 6: SCATTERPLOT
        figure6 = px.scatter(data_frame=numeric_df, x=Variables_x, y=Variable_y,
                     title= 'Modelo Lineal Múltiple')
        st.plotly_chart(figure6)
        
 ####################################################################################

   # CONTENIDO DE LA VISTA 3
if View == "Regresión No Lineal":
    #REGRESIÓN NO LINEAL 
    #Generamos la lista de variables numéricas
    numeric_df = df.select_dtypes(['float','int'])  #Devuelve Columnas
    df_men= df[df["Sex"] == "male"]
    df_women=df[df["Sex"] == "female"]
    Lista_num= numeric_df.columns                   #Devuelve lista de Columnas numéricas
    #Select box
    Variable_y= st.sidebar.selectbox(label= "Variable objetivo (Y)", options= Lista_num)
    Variable_x= st.sidebar.selectbox(label= "Variable independiente del modelo No lineal (X)", options= Lista_num)   
    #Lista de modelos
    Lista_mod=["Función cuadrática", "Función exponencial"]
    #Select box de modelos
    Modelo= st.sidebar.selectbox(label= "Modelos No Lineales", options= Lista_mod)  

    #Generamos los encabezados para el dashboard
    st.title("Regresión No Lineal")  
        
        
    #Generamos el diseño del Layout deseado
    # Fila 1
    Contenedor_A, Contenedor_B = st.columns(2)
    with Contenedor_A: 
        st.write("Correlación No lineal Hombres")
        #Redefinimos las variables 
        x= df_men[Variable_x]
        y= df_men[Variable_y]
        
        #Declaración de MODELOS
        if Modelo == "Función cuadrática":
            def  func1 (x, a, b, c):
                return a*x**2 + b*x + c
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func1, x, y)
            # Predicciones
            y_pred = func1(x, *parametros)
        
        if Modelo == "Función exponencial":
            def func2(x, a, b, c):
                return a*np.exp(-b*x)+c
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func2, x, y)
            # Predicciones
            y_pred = func2(x, *parametros)

        #Corroboramos cual es el coeficiente de Determinación de nuestro modelo
        coef_Deter = r2_score(y, y_pred) 
        #Corroboramos cual es el coeficiente de Correlación de nuestro modelo
        coef_Correl=np.sqrt(coef_Deter)
        #Mostramos el dataset
        st.write(coef_Correl)
        
        #GRAPH 7: SCATTERPLOT
        figure7 = px.scatter(data_frame=df_men, x=Variable_x, y=Variable_y, 
                     title= 'Modelo No Lineal para Hombres')
        st.plotly_chart(figure7)
        

    with Contenedor_B: 
        st.write("Correlación No lineal Mujeres")
        #Redefinimos las variables 
        x= df_women[Variable_x]
        y= df_women[Variable_y]
               #Declaración de MODELOS
        if Modelo == "Función cuadrática":
            def  func1 (x, a, b, c):
                 return a*x**2 + b*x + c
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func1, x, y)
            # Predicciones
            y_pred = func1(x, *parametros)
    
        if Modelo == "Función exponencial":
            def func2(x, a, b, c):
                return a*np.exp(-b*x)+c
            #Ajustamos los parámetros de la función curve_fit
            parametros, covs= curve_fit(func2, x, y)
            # Predicciones
            y_pred = func2(x, *parametros)

 
        #Corroboramos cual es el coeficiente de Determinación de nuestro modelo
        coef_Deter = r2_score(y, y_pred) 
        #Corroboramos cual es el coeficiente de Correlación de nuestro modelo
        coef_Correl=np.sqrt(coef_Deter)
        #Mostramos el dataset
        st.write(coef_Correl)
        
        #GRAPH 7: SCATTERPLOT
        figure8 = px.scatter(data_frame=df_women, x=Variable_x, y=Variable_y, 
                     title= 'Modelo No Lineal para Mujeres')
        st.plotly_chart(figure8)
   
   
   
   ##########################################################################################3
        # CONTENIDO DE LA VISTA 4
if View == "Regresión Logística":
    #REGRESIÓN NO LINEAL 
    #Generamos la lista de variables numéricas
    numeric_df = df.select_dtypes(['float','int'])  #Devuelve Columnas
    Lista_num= numeric_df.columns                   #Devuelve lista de Columnas numéricas
    Lista_dicot= ['Survived','Sex']                  #Devuelve lista de Columnas dicotómicas
    #Select box
    Variable_y= st.sidebar.selectbox(label= "Variable dependiente (y)", options= Lista_dicot)
    #Multi-select box
    Variables_x= st.sidebar.multiselect(label="Variables independientes del modelo logístico (X)", options= Lista_num,
                                        default=['Fare'])  

    #Generamos los encabezados para el dashboard
    st.title("Regresión Logística")  
        


    #Generamos el diseño del Layout deseado
    # Fila 1
    Contenedor_A, Contenedor_B = st.columns(2)
    with Contenedor_A: 
        st.write("Correlación Logística General")
        #Redefinimos las variables 
        X= df[Variables_x]
        y= df[Variable_y]
        
        #Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
        
        #Se escalan todos los datos
        escalar = StandardScaler()
        
        #Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
        X_train = escalar.fit_transform(X_train)
        X_test = escalar.transform(X_test)

        #Definimos el algoritmo a utilizar
        from sklearn.linear_model import LogisticRegression
        algoritmo = LogisticRegression()
        
        #Entrenamos el modelo
        algoritmo.fit(X_train, y_train)
        
        #Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
        X_train = escalar.fit_transform(X_train)
        X_test = escalar.transform(X_test)
        
        #Realizamos una predicción
        y_pred = algoritmo.predict(X_test) 
        
        #Verifico la matriz de Confusión
        matriz = confusion_matrix(y_test, y_pred)

        # Etiquetas de los ejes
        clases=  np.unique(df[Variable_y])
        labels = [clases[0], clases[1]]

        # Crear el heatmap
        figure9 = go.Figure(data=go.Heatmap(
            z=matriz,
            x=labels,
            y=labels,
            hoverinfo="z",
            colorscale="Oranges",
            showscale=True,
            zmin=0
        ))

        # Añadir anotaciones con TP, TN, FP, FN
        annotations = []
        for i in range(matriz.shape[0]):
            for j in range(matriz.shape[1]):
                valor = matriz[i, j]
                if i == 0 and j == 0:
                    texto = f'TP: {valor}'
                elif i == 0 and j == 1:
                    texto = f'FP: {valor}'
                elif i == 1 and j == 0:
                    texto = f'FN: {valor}'
                elif i == 1 and j == 1:
                    texto = f'TN: {valor}'
                annotations.append(
                    dict(
                        x=labels[j],
                        y=labels[i],
                        text=texto,
                        showarrow=False,
                        font=dict(color="white" if valor > matriz.max()/2 else "black")
                    )
                )

        figure9.update_layout(
            title='Confusion Matrix',
            xaxis_title="Predicted",
            yaxis_title="Actual",
            annotations=annotations,
            width=500,
            height=500
        )

        # Mostrar en Streamlit
        st.plotly_chart(figure9)

        #Calculo la exactitud del modelo
        from sklearn.metrics import accuracy_score
        exactitud = accuracy_score(y_test, y_pred)
        #Mostramos la exactitud del modelo
        st.write("Exactitud del Modelo")
        st.write(exactitud)
        #Calculo la precisión del modelo
        from sklearn.metrics import precision_score
        precision_0 = precision_score(y_test, y_pred, average="binary", pos_label=clases[0])
        st.write("Precisión de la etiqueta:", clases[0])
        st.write(precision_0)
        precision_1 = precision_score(y_test, y_pred, average="binary", pos_label=clases[1])
        st.write("Precisión de la etiqueta:", clases[1])
        st.write(precision_1)
        
 ####################################################################################
